# ***Molecular Doking (AutoDock Vina)***

This notebook is for analyzing molecular docking simulaitons.

# **1. Environment ready for molecular docking**

  1. AutoDock Vina (https://vina.scripps.edu/): Program / Molecular docking, vitrual screening
  2. Biopython (https://biopython.org/): Library / Working with biological sequences and data
  3. Condacolab (https://github.com/conda-incubator/
condacolab): Module / setting up Conda environments in Google Colab
  4. MGLtools (https://ccsb.scripps.edu/mgltools/): suite of programs (including libraries and executables) / Preparing protein and ligand structures for docking, particularly with AutoDock Vina
  5. OpenBabel (https://github.com/openbabel/openbabel): Toolkit (library , command-line tools) / Chemical file format conversion, molecular structure manipulation
  6. PLIP (https://plip-tool.biotec.tu-dresden.de/plip-web/plip/index): Tool (web server, command-line) / Protein-ligand interaction analysis, visualization
  7. Py3Dmol (https://pypict/py3Dmol).org/proje: Library / 3D molecular visualization
  8. Pdb2pqr: Tool (command-line) / Preparing protein structures for electrostatics calculations
  9. Kora(Github): Module / Utilities for cheminformatics, bioinformatics in Colab
  10. Rdkit(https://www.rdkit.org): Library / Cheminformatics and machine learning for working with chemi structures and data

In [ ]:
#@title **Install packages**
# Install condacolab first

!pip install -q condacolab
import condacolab

# In case using GPU, if not using attach '#""
#%load_ext cudf.pandas

condacolab.install_mambaforge()

✨🍰✨ Everything looks OK!


Mambaforge has been sunset. It is now identical to Miniforge. Installing Miniforge...


In [ ]:
!rm -r /content/sample_data
!wget https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.5/vina_1.2.5_linux_x86_64 -O vina
!chmod u+x vina

!pip install py3dmol==2.0.3

!mamba install -c conda-forge -c bioconda \
rdkit=2024.03.5 \
plip \
biopython=1.85 \
openbabel=3.1.1 \
xlsxwriter=3.0.3 \
pdb2pqr \
mgltools=1.5.7 \
numpy=1.26 \
pybel \
-y

print(f" + Success install")

--2025-10-04 14:05:29--  https://github.com/ccsb-scripps/AutoDock-Vina/releases/download/v1.2.5/vina_1.2.5_linux_x86_64
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/258054635/b208f84f-df05-4575-9991-2190698c7914?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-10-04T15%3A00%3A08Z&rscd=attachment%3B+filename%3Dvina_1.2.5_linux_x86_64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-10-04T14%3A00%3A01Z&ske=2025-10-04T15%3A00%3A08Z&sks=b&skv=2018-11-09&sig=3n5KkPODZFWPyVLunJz2HO1YGfQH09KLeXtiS9gnuF0%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc1OTU4NzAyOSwibmJmIjoxNzU5NTg2NzI5LCJwYXRoIjoicmVsZWFzZ

In [ ]:
%alias vina /content/vina
from typing import BinaryIO

# Internal Modules
import sys
import os
import glob # finding files matching a specific pattern.
import time
import shutil # high-level file operations (like copying and moving).
import math
import urllib.request # opening and reading URLs.
import ast

# External modules
import py3Dmol
import plip
import pdb2pqr
import Bio

# Data-related
import numpy as np
import pandas as pd

# Google Colab-related
from google.colab import drive, files
from tqdm.notebook import tqdm # provides progress bars for long-running operations

# Docking-related
from rdkit import Chem, RDLogger
from rdkit.Chem import rdFMCS, AllChem, Draw
from rdkit.Chem import PandasTools
from Bio.PDB import PDBIO, PDBParser
from openbabel import pybel

# Binding interaction-related
from plip.exchange.report import BindingSiteReport
from plip.structure.preparation import PDBComplex

print(f'+ Import completed')

+ Import completed


# **Create folders**

In [ ]:
#@markdown Enter a **< Job Name >** without space.\
#@markdown This creates a folder for protein, ligand, experimental, and docking.

Job_name = "MMP9_4WZV_1" #@param {type:"string"}
assert not Job_name == "", "Do not leave this blank."
assert not any(c == "/" or c == "." for c in Job_name), "Disallowed characters."

DIR = os.getcwd()
WRK_DIR = os.path.join(DIR, Job_name)
PRT_FLD = os.path.join(WRK_DIR, "PROTEIN")
LIG_FLD = os.path.join(WRK_DIR, "LIGAND")
EXP_FLD = os.path.join(WRK_DIR, "EXPERIMENTAL")
DCK_FLD = os.path.join(WRK_DIR, "DOCKING")
INT_FLD = os.path.join(WRK_DIR, "INTERACTION")

folders = [WRK_DIR, PRT_FLD, LIG_FLD, EXP_FLD, DCK_FLD, INT_FLD]

for f in folders:
    if os.path.exists(f):
        print(f"+ {os.path.basename(f)} folder already exists")
    if not os.path.exists(f):
        os.mkdir(f)
        print(f"+ {os.path.basename(f)} folder created")

+ MMP9_4WZV_1 folder created
+ PROTEIN folder created
+ LIGAND folder created
+ EXPERIMENTAL folder created
+ DOCKING folder created
+ INTERACTION folder created


# **Setting of utilities**
  - This creates important variables and functions that will be utilized on docking study.

In [ ]:
COLORS = ["red", "orange", "yellow", "lime", "green", "cyan", "teal", "blue",
          "violet", "purple", "pink", "gray", "brown", "white", "black"]

BOND_DICT = {"hydrophobic": ["0x59e382", "GREEN"],
             "hbond": ["0x59bee3", "LIGHT BLUE"],
             "waterbridge": ["0x4c4cff", "BLUE"],
             "saltbridge": ["0xefd033", "YELLOW"],
             "pistacking": ["0xb559e3", "PURPLE"],
             "pication": ["0xe359d8", "VIOLET"],
             "halogen": ["0x59bee3", "LIGHT BLUE"],
             "metal": ["0xe35959", "ORANGE"]}

class Hide:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, "w")
    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

# Biological assembly detector and extractor
def chain_profilier(inputFile):
    parser = PDBParser()
    structure = parser.get_structure("X", inputFile)
    chain_obj = [s for s in structure.get_chains()]
    chain_IDs = [c.get_id() for c in chain_obj]
    chain_len = len(chain_obj)
    return [chain_len, chain_IDs, chain_obj]

def seperate_chain(inputFile):
    io = PDBIO()
    name = os.path.basename(inputFile)[:-4]
    dir = os.path.dirname(inputFile)
    chain_info = chain_profilier(inputFile)
    chain_len, chain_IDs, chain_obj = chain_info[0], chain_info[1], chain_info[2]
    print(f"+ Chains detected: {chain_len} ({', '.join(chain_IDs)})")
    for ID,chain in zip(chain_IDs, chain_obj):
        chain_pdb = name + "_" + ID + ".pdb"
        chain_pdb_file = os.path.join(dir, chain_pdb)
        io.set_structure(chain)
        io.save(chain_pdb_file)
        print(f"+ {chain_pdb} ==> {os.path.basename(dir)} folder")
# Gridbox methods
def get_atom_coordinate(inputE):
    mol = Chem.MolFromPDBFile(inputE)
    molAtom = mol.GetAtoms()
    molConf = mol.GetConformer()
    xcoor = [molConf.GetAtomPosition(c).x for c in range(len(molAtom))]
    ycoor = [molConf.GetAtomPosition(c).y for c in range(len(molAtom))]
    zcoor = [molConf.GetAtomPosition(c).z for c in range(len(molAtom))]
    return [xcoor, ycoor, zcoor]

def labox(coorList, scale=2):
    X, Y, Z = coorList[0], coorList[1], coorList[2]
    range = [[min(X), max(X)],
             [min(Y), max(Y)],
             [min(Z), max(Z)]]
    center = [np.round(np.mean(range[0]), 3),
              np.round(np.mean(range[1]), 3),
              np.round(np.mean(range[2]), 3)]
    size = [np.round(abs(range[0][0]-range[0][1])*scale, 3),
            np.round(abs(range[1][0]-range[1][1])*scale, 3),
            np.round(abs(range[2][0]-range[2][1])*scale, 3)]
    return [center, size]

def eboxsize(coorList, scale=1):
    gyBoxRatio = 0.23
    X, Y, Z = coorList[0], coorList[1], coorList[2]
    center = [np.round(np.mean(X), 3),
              np.round(np.mean(Y), 3),
              np.round(np.mean(Z), 3),]
    dist = [(x-center[0])**2 +
            (y-center[1])**2 +
            (z-center[2])**2
            for x,y,z in zip(X,Y,Z)]
    size = [np.round((math.sqrt(np.sum(dist)/len(X))/gyBoxRatio)*scale, 3)]*3
    return [center, size]

def eboxsize_mod(coorList, scale=1):
    gyBoxRatio = 0.23
    X, Y, Z = coorList[0], coorList[1], coorList[2]
    range = [[min(X), max(X)],
             [min(Y), max(Y)],
             [min(Z), max(Z)]]
    center = [np.round(np.mean(range[0]), 3),
              np.round(np.mean(range[1]), 3),
              np.round(np.mean(range[2]), 3)]
    dist = [(x-center[0])**2 +
            (y-center[1])**2 +
            (z-center[2])**2
            for x,y,z in zip(X,Y,Z)]
    size = [np.round((math.sqrt(np.sum(dist)/len(X))/gyBoxRatio)*scale, 3)]*3
    return [center, size]

def autodock_grid(coorList, scale=1):
    min_len = 22.5
    X, Y, Z = coorList[0], coorList[1], coorList[2]
    range = [[min(X), max(X)],
             [min(Y), max(Y)],
             [min(Z), max(Z)]]
    center = [np.round(np.mean(range[0]), 3),
              np.round(np.mean(range[1]), 3),
              np.round(np.mean(range[2]), 3)]
    size = [min_len]*3
    return [center, size]

# Binding interactions profiler
def interaction_profiler(inputPL):
    protlig = PDBComplex()
    INTER = {}
    INTER_TYPE = []
    ID = os.path.basename(inputPL[:-7])
    CSV_Ifile = os.path.join(INT_FLD, f"{ID[:-2]}/{ID}_inter.csv")
    COL = ["RESNR", "RESTYPE", "RESCHAIN", "RESNR_LIG", "RESTYPE_LIG",
           "RESCHAIN_LIG", "DIST", "LIGCARBONIDX", "PROTCARBONIDX", "LIGCOO",
           "PROTCOO"]
    INTER_DF = pd.DataFrame(columns=COL)

    protlig.load_pdb(inputPL)
    LIG = protlig.ligands[0]
    protlig.characterize_complex(LIG)

    BD_keys = BOND_DICT.keys()
    BSI_key = f":".join([str(x) for x in LIG.members[0]])
    BSI_val = protlig.interaction_sets[BSI_key]
    BSR = BindingSiteReport(BSI_val)
    INTER[BSI_key] = {k: [getattr(BSR, f"{k}_features")] +
                      getattr(BSR, f"{k}_info")
                      for k in tuple(BD_keys)}
    for BD_key in list(BD_keys):
        DF = pd.DataFrame.from_records(
            INTER[BSI_key][BD_key][1:],
            columns=INTER[BSI_key][BD_key][0])
        if not DF.empty:
            INTER_TYPE.extend([BD_key.upper()]*len(DF))
            INTER_DF = pd.concat([INTER_DF, DF], ignore_index=True)
    INTER_DF = INTER_DF.assign(BOND=INTER_TYPE)
    INTER_DF.reset_index(drop=True, inplace=True)
    INTER_DF.to_csv(CSV_Ifile, index=False)

def view_interaction(inputCSV, result="summary"):
    DIST_CALC = []
    COLOR = []
    MIDCOO = []

    def parse_coordinates(coord_string):
      print(f"Parsing coordinate string: {coord_string}") # Debug print
      # Remove brackets and split by comma
      coords = coord_string.strip('[]').split(',')
      # Convert each part to float
      return [float(c.strip()) for c in coords]

    interaction = pd.read_csv(inputCSV, converters={
        "LIGCOO": parse_coordinates,
        "PROTCOO": parse_coordinates,
        "BOND": lambda x: x.lower()})

    for LC, PC, BT in zip(interaction["LIGCOO"], interaction["PROTCOO"],
                          interaction["BOND"]):
        COLOR.append(BOND_DICT[BT][1])
        p1 = np.array([LC[0], LC[1], LC[2]])
        p2 = np.array([PC[0], PC[1], PC[2]])
        squared_dist = np.sum((p1-p2)**2, axis=0)
        dist = np.round(np.sqrt(squared_dist) ,2)
        DIST_CALC.append(dist)
        mid_x = np.round((LC[0] + PC[0]) / 2, 2)
        mid_y = np.round((LC[1] + PC[1]) / 2, 2)
        mid_z = np.round((LC[2] + PC[2]) / 2, 2)
        p_mid = (mid_x, mid_y, mid_z)
        MIDCOO.append(p_mid)
    interaction["BOND"] = interaction["BOND"].str.upper()
    interaction["COLOR"] = COLOR
    interaction["MIDCOO"] = MIDCOO
    interaction["DIST_CALC"] = DIST_CALC
    interaction["RESNAME"] = interaction["RESTYPE"] + interaction["RESNR"].astype(str)

    summary = ["RESNR", "RESTYPE", "DIST_CALC", "BOND", "COLOR"]
    py3dmol = ["RESNR", "DIST_CALC", "LIGCOO", "PROTCOO", "MIDCOO", "BOND"]
    if result == "summary":
        df = interaction[summary]
    if result == "py3Dmol":
        df = interaction[py3dmol]
    return df

# Docking result clustering
def get_RMSD(ref, target):
    distances = []
    maxSubStr = rdFMCS.FindMCS([ref, target])
    a = ref.GetSubstructMatch(Chem.MolFromSmarts(maxSubStr.smartsString))
    b = target.GetSubstructMatch(Chem.MolFromSmarts(maxSubStr.smartsString))
    for atomA,atomB in list(zip(a, b)):
        pos_A = ref.GetConformer().GetAtomPosition(atomA)
        pos_B = target.GetConformer().GetAtomPosition(atomB)
        coord_A = np.array((pos_A.x, pos_A.y, pos_A.z))
        coord_B = np.array((pos_B.x, pos_B.y, pos_B.z))
        dist_numpy = np.linalg.norm(coord_A - coord_B)
        distances.append(dist_numpy)
    rmsd = np.round(math.sqrt(1/len(distances)*sum([i*i for i in distances])),3)
    return rmsd

def get_score(inputL, inputE, result="full"):
    ID = os.path.basename(inputL)[:-11]
    df = PandasTools.LoadSDF(inputL)
    df["ID"] = [ID + "_" + df.loc[i,"POSE"] for i in df.index]
    df["RMSD_EL"] = [get_RMSD(inputE, df.loc[i, "ROMol"]) for i in df.index]
    df["SCORE"] = pd.to_numeric(df["SCORE"])
    df["RMSD_EL"] = pd.to_numeric(df["RMSD_EL"])
    df["RMSD_LB_BP"] = pd.to_numeric(df["RMSD_LB_BP"])
    df["RMSD_UB_BP"] = pd.to_numeric(df["RMSD_UB_BP"])
    first = ["ID", "SCORE", "RMSD_EL"]
    full = ["ID", "POSE", "SCORE", "RMSD_EL", "RMSD_LB_BP", "RMSD_UB_BP"]
    if result == "first":
        return df[first].iloc[:1]
    if result == "full":
        return df[full]

# Py3Dmol model viewer
def VIEW_PROT(inputP, model="Cartoon", color="white", focusRes="",
              focusResColor="white", addStick=False, addLine=False,
              showChain=False, showResLabel=False, showVDW=False,
              outline=False):
    # Variables
    mview = py3Dmol.view(width=960, height=640)
    if model in "none":
        model = cType = color = ""
    if model in "cartoon":
        cType = "color"
    if model in ("line", "stick"):
        cType, color = "colorscheme", color + "Carbon"

    # Protein Model
    count = 0
    prot_model = count
    print(f"+ Showing {os.path.basename(inputP)}")
    mol = open(inputP, "r").read()
    mview.addModel(mol, "pdb")
    mview.setStyle(
        {"model": prot_model},
        {model: {cType: color}})

    # Show chains
    chainLen = chain_profilier(inputP)[0]
    if showChain and chainLen > 1:
        for n,c in zip(range(chainLen), COLORS):
            mview.setStyle(
                {"and": [{"model": prot_model}, {"chain": chr(65+n)}]},
                {model: {cType: c if model == "cartoon" else c + "Carbon"}})
            mview.addLabel(
                f"Chain {chr(65+n)}",
                {"fontColor": c, "backgroundOpacity": 0.7, "alignment": "topLeft"},
                {"and": [{"model": prot_model}, {"chain": chr(65+n)}]})

    # Focus RES
    if focusRes != "":
        res = focusRes.split(",")
        mview.addStyle(
            {"and": [{"model": prot_model}, {"resi": res}]},
            {"stick": {"colorscheme": focusResColor + "Carbon", "radius": 0.15}})
        # Label focused RES
        if showResLabel:
            mview.addResLabels(
                {"and": [{"model": prot_model}, {"resi": res}]},
                {"alignment": "bottomLeft",
                 "showBackground": False,
                 "inFront": True,
                 "fixed": False,
                 "fontSize": 14,
                 "fontColor": "0x000000",
                 "screenOffset": {"x": 15, "y": 15}})

    # Show VDW surface
    if showVDW:
        mview.addSurface(
            py3Dmol.VDW,
            {"color": "white", "opacity": 0.7},
            {"model": prot_model})

    # Show outline
    if outline:
        mview.setViewStyle(
            {"style": "outline",
            "color": "black",
            "width": 0.1})

    print(f"")
    mview.setBackgroundColor("0xFFFFFF")
    mview.center({"model": prot_model})
    mview.show()

def VIEW_ELIG(inputE, showAtomLabel=False, outline=False):
    # Variable
    mview = py3Dmol.view(width=960, height=640)

    # Experimental ligand model
    count = 0
    elig_model = count
    print(f"+ Showing {os.path.basename(inputE)}")
    mol = open(inputE, "r").read()
    mview.addModel(mol, "pdb")
    mview.setStyle(
        {"model": elig_model},
        {"stick": {"colorscheme": "lightGrayCarbon"}})

    # Label all atoms
    if showAtomLabel:
        mview.addPropertyLabels(
            "atom",
            {"model": elig_model},
            {"backgroundOpacity": 0.7, "inFront": True})

    # Show outline
    if outline:
        mview.setViewStyle(
            {"style": "outline",
            "color": "black",
            "width": 0.1})

    print(f"")
    mview.setBackgroundColor("0xFFFFFF")
    mview.zoomTo({"model": elig_model})
    mview.show()

def VIEW_LIG(inputL, showHs=False, showAtomLabel=False, outline=False):
    # Variable
    mview = py3Dmol.view(width=960, height=640)

    # Ligand model
    count = 0
    lig_model = count
    print(f"+ Showing {os.path.basename(inputL)}")
    smi = open(inputL, "r").read()
    mol = Chem.MolFromSmiles(smi)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)

    # Remove Hs
    if not showHs:
        mol = Chem.RemoveHs(mol)
    mblock = Chem.MolToMolBlock(mol)
    mview.addModel(mblock, "mol")
    mview.setStyle(
        {"model": lig_model},
        {"stick": {"colorscheme": "lightGrayCarbon"}})

    # Label all atoms
    if showAtomLabel:
        mview.addPropertyLabels(
            "atom",
            {"model": lig_model},
            {"backgroundOpacity": 0.7, "inFront": True})

    # Show outline
    if outline:
        mview.setViewStyle(
            {"style": "outline",
            "color": "black",
            "width": 0.1})

    print(f"")
    mview.setBackgroundColor("0xFFFFFF")
    mview.center({"model": lig_model})
    mview.show()

def VIEW_GRID(inputP, inputE, focusRes, center, size=[10, 10, 10]):
    # Variable
    mview = py3Dmol.view(width=960, height=640)

    # Grid box
    bxi, byi, bzi = center[0], center[1], center[2]
    bxf, byf, bzf = size[0], size[1], size[2]
    print(f"+ Center: X {center[0]}  Y {center[1]}  Z {center[2]}")
    print(f"+ Size: W {size[0]}  H {size[1]}  D {size[2]}")
    mview.addBox(
        {"center":{"x":bxi, "y":byi, "z":bzi},
        "dimensions": {"w":bxf, "h":byf, "d":bzf},
        "color": "skyBlue", "opacity": 0.7})

    # Protein model
    count = 0
    prot_model = count
    print(f"+ Showing {os.path.basename(inputP)}")
    molA = open(inputP, "r").read()
    mview.addModel(molA, "pdb")
    mview.setStyle(
        {"model": prot_model},
        {"cartoon": {"color": "white"}})

    # Experimental ligand model
    count += 1
    elig_model = count
    print(f"+ Showing {os.path.basename(inputE)}")
    molB = open(inputE, "r").read()
    mview.addModel(molB, "pdb")
    mview.setStyle(
        {"model": elig_model},
        {"stick": {"colorscheme": "greenCarbon"}})

    # Focus RES
    if focusRes != "":
        res = focusRes.split(",")
        mview.addStyle(
            {"and": [{"model": prot_model}, {"resi": res}]},
            {"stick": {"colorscheme": "orangeCarbon", "radius": 0.15}})
        mview.addResLabels(
            {"and": [{"model": prot_model}, {"resi": res}]},
            {"alignment": "bottomLeft",
            "showBackground": False,
            "inFront": True,
            "fixed": False,
            "fontSize": 14,
            "fontColor": "0x000000",
            "screenOffset": {"x": 15, "y": 15}})

    print(f"")
    mview.setBackgroundColor("0xFFFFFF")
    mview.center({"model": elig_model})
    mview.show()

def VIEW_PILE(inputP, inputPP, inputL, inputE, inputCSV, model="Cartoon",
              color="white", focusRes="", focusResColor="white",
              showInter=False, showSubunit=False, showResLabel=False,
              showVDW=False, showPartProt=False, showExpLig=False,
              showLig=False, showAllPose=False, showBestPose=False,
              outline=False):
    # Variables
    mview = py3Dmol.view(width=960, height=640)
    if model in "none":
        model = cType = color = ""
    if model in "cartoon":
        cType = "color"
    if model in ("line", "stick"):
        cType, color = "colorscheme", color + "Carbon"

    # Protein model
    count = 0
    prot_model = count
    print(f"+ Showing {os.path.basename(inputP)}")
    molA = open(inputP, "r").read()
    mview.addModel(molA, "pdb")
    mview.setStyle(
        {"model":prot_model},
        {model: {cType: color}})

  # Show chains
    chainLen = chain_profilier(inputP)[0]
    if showSubunit and chainLen > 1:
        for n, c in zip(range(chainLen), COLORS):
            mview.setStyle(
                {"and": [{"model": prot_model}, {"chain": chr(65+n)}]},
                {model: {cType: c if model == "cartoon" else c + "Carboon"}})
            mview.addLabel(
                f"Subunit {chr(65+n)}",
                {"fontColor": c, "backgroundOpacity": 0.7, "alignment": "topLeft"},
                {"and": [{"model": prot_model}, {"chain": chr(65+n)}]})

    # Focus RES
    res = focusRes.split(',')
    if focusRes != "":
        mview.addStyle(
            {"and": [{"model": prot_model}, {"resi": res}]},
            {"stick": {"colorscheme": focusResColor + "Carbon", "radius": 0.15}})
        # Label focused RES
        if showResLabel:
            mview.addResLabels(
                {"and": [{"model": prot_model}, {"resi": res}]},
                {"alignment": "bottomLeft",
                 "showBackground": False,
                 "inFront": True,
                 "fixed": False,
                 "fontSize": 14,
                 "fontColor": "0x000000",
                 "screenOffset": {"x": 15, "y": 15}})

    # Interactions
    if showInter:
        interaction = view_interaction(inputCSV, result="py3Dmol")
        RESNR = interaction["RESNR"]
        DIST_CALC = interaction["DIST_CALC"]
        LIGCOO = interaction["LIGCOO"]
        PROTCOO = interaction["PROTCOO"]
        MIDCOO = interaction["MIDCOO"]
        BOND = interaction["BOND"]

        inter_res = list(set(RESNR) - set(res))
        mview.addStyle(
            {"and": [{"model": prot_model}, {"resi": inter_res}]},
            {"stick": {"colorscheme": "whiteCarbon", "radius": 0.15}})
        mview.addResLabels(
            {"and": [{"model": prot_model}, {"resi": inter_res}]},
            {"alignment": "bottomLeft", "showBackground": False,
                "inFront": True, "fixed": False, "fontSize": 14,
                "fontColor": "0x000000", "screenOffset": {"x": 15, "y": 15}})

        for RN,DC,LC,PC,MC,BT in zip(RESNR, DIST_CALC, LIGCOO, PROTCOO, MIDCOO, BOND):
            BT = BT.lower()
            mview.addCylinder(
                {"start": {"x": LC[0], "y": LC[1], "z": LC[2]},
                    "end": {"x": PC[0], "y": PC[1], "z": PC[2]},
                    "radius": 0.05, "fromCap": 1, "toCap": 1,
                    "color": BOND_DICT[BT][0], "dashed": True})
            mview.addLabel(
                str(DC) + " Å",
                {"position": {"x": MC[0], "y": MC[1], "z": MC[2]},
                    "alignment": "bottomLeft", "inFront": False, "fixed": False,
                    "backgroundColor": BOND_DICT[BT][0], "fontSize": 10,
                    "screenOffset": {"x": 5, "y": 5}})

    # Show VDW surface
    if showVDW:
        mview.addSurface(
            py3Dmol.VDW,
            {"color": "white", "opacity": 0.4},
            {"model": prot_model})

    # Partner protein model
    if showPartProt and os.path.exists(inputPP):
        count += 1
        pprot_model = count
        print(f"+ Showing {os.path.basename(inputPP)} (yellow)")
        molB = open(inputPP, "r").read()
        mview.addModel(molB, "pdb")
        mview.setStyle(
            {"model": pprot_model},
            {"cartoon": {"color": "yellow"}})
        mview.addStyle(
            {"model": pprot_model},
            {"stick": {"colorscheme": "yellowCarbon"}})

    # Experimental ligand model
    if showExpLig:
        count += 1
        elig_model = count
        print(f"+ Showing {os.path.basename(inputE)} (gray)")
        molC = open(inputE, "r").read()
        mview.addModel(molC, "pdb")
        mview.setStyle(
            {"model": elig_model},
            {"stick": {"color": "gray"}})

    # Ligand model
    if showLig:
        count += 1
        lig_model = count
        print(f"+ Showing {os.path.basename(inputL)} (orange)")
        molD = open(inputL, "r").read()
        mview.addModel(molD, "pdb")
        mview.setStyle(
            {"model": lig_model},
            {"stick": {"colorscheme": "orangeCarbon"}})

    # Show poses of selected ligand
    if showAllPose:
        pose = sorted([ os.path.join(os.path.dirname(inputL), f) for f in os.listdir(os.path.dirname(inputL)) if f.endswith(".pdb") ])
        pose.remove(inputL[len(DCK_FLD)+1:]) if inputL[len(DCK_FLD)+1:] in pose else None
        for f in pose:
            count += 1
            pose_model = count
            fs = "".join(f)
            mol5 = open(os.path.join(DCK_FLD, fs), "r").read()
            mview.addModel(mol5, "pdb")
            mview.setStyle(
                {"model": pose_model},
                {"stick": {"color": "blue", "opacity": 0.5, "radius": 0.2}})

    # Show all best poses of docked ligands
    if showBestPose:
        pose = sorted([ f + "_1.pdb" for f in os.listdir(DCK_FLD) if os.path.isdir(os.path.join(DCK_FLD, f)) ])
        pose.remove(inputL[len(DCK_FLD)+1:]) if inputL[len(DCK_FLD)+1:] in pose else None
        for f in pose:
            count += 1
            pose_model = count
            fs = "".join(f)
            mol6 = open(os.path.join(DCK_FLD, f"{fs[:-6]}/{fs}"), "r").read()
            mview.addModel(mol6, "pdb")
            mview.setStyle(
                {"model": pose_model},
                {"stick": {"color": "green", "opacity": 0.5, "radius": 0.2}})

    if outline:
        mview.setViewStyle(
            {"style": "outline", "color": "black", "width": 0.1})

    print(f"")
    mview.setBackgroundColor("0xFFFFFF")
    mview.center({"model": lig_model})
    mview.enableFog(True)
    mview.show()

#2. **Preparing the Receptor molecule for Docking**
  - Download PDB foramt files using online DB("RSCB PDB")**
  * Choosing target protein (higher resolution < 2.5 Å, binding ligand, non-essential metal ions and species)
    - When to keep ions, Check original PDB paper or UniProt functional annotation. and Binding site.
  - Clean protein structure and extract subunits
  - Remove Water molecules (HOH)
  - Addition of the Polar Hydrogens
  - Assign Protonation States (pH 7.4)
  - Addition of the Gasteiger Partial Charge
  - Convert PDB into PDBQT

In [ ]:
# @title **Download protein PDB file**
# @markdown Enter the ** ( Accession ID )** to download the protein.

Accession_ID = '4WZV' # @param {type: 'string'}

PDB_pdb = Accession_ID.upper() + '.pdb'
PDB_pdb_pFile = os.path.join(PRT_FLD, PDB_pdb)
Accession_ID_url = 'http://files.rcsb.org/download/' + PDB_pdb

urllib.request.urlretrieve(f"http://files.rcsb.org/download/{PDB_pdb}", PDB_pdb_pFile)
print(f'+ PDB downloaded: {PDB_pdb} > PROTEIN folder')
print(f'+ RCSB PDB link: https://www.rcsb.org/structure/{Accession_ID}')


+ PDB downloaded: 4WZV.pdb > PROTEIN folder
+ RCSB PDB link: https://www.rcsb.org/structure/4WZV


In [ ]:
#@title **Extract protein and delete the heteroatom**
#@markdown This generate **clean** protein structure for molecular docking.\
#@markdown The protein chains will be extracted into separate files.
#@markdown delete the "HETATM"

PDB_prot = Accession_ID + "_prot"
PDB_prot_pdb = PDB_prot + ".pdb"
PDB_prot_pdb_pFile = os.path.join(PRT_FLD, PDB_prot_pdb)

with open(PDB_prot_pdb_pFile, "w") as o, open(PDB_pdb_pFile, "r") as i:
    for line in i:
        row = line.split()
        if row[0] == "ATOM":
            o.write(line)
        elif row[0] == "TER":
            o.write("TER\n")

print("Successfully created")

Successfully created


In [ ]:
#@title **Display the protein into 3D structure** { run: "auto" }
#@markdown Enter the ** ( Protein ) ** to be viewed.
#@markdown %Ignoring waring "TER"%

View = "4WZV" #@param {type:"string"}
Model = "none" #@param ["none", "line", "stick",  "cartoon"]
Model_colour = "white" #@param ["red", "orange", "yellow", "green", "blue", "violet", "purple", "white"]
Focus_residues = "" #@param {type:"string"}
Residues_colour = "red" #@param ["red", "orange", "yellow", "green", "blue", "violet", "purple", "white"]
Show_all_chains = True #@param {type:"boolean"}
Show_residues_label = True #@param {type:"boolean"}
Show_VDW_surface = True #@param {type:"boolean"}
View_outline = True #@param {type:"boolean"}
pview_Pfile = os.path.join(PRT_FLD, f"{View}.pdb")

VIEW_PROT(inputP=pview_Pfile,
          model=Model,
          color=Model_colour,
          focusRes=Focus_residues,
          focusResColor=Residues_colour,
          showChain=Show_all_chains,
          showResLabel=Show_residues_label,
          showVDW=Show_VDW_surface,
          outline=View_outline)

2025-10-04 14:07:08,885 [WARNING] [warnings.py:110] py.warnings: /usr/local/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3469.
  warnings.warn(

2025-10-04 14:07:08,887 [WARNING] [warnings.py:110] py.warnings: /usr/local/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3542.
  warnings.warn(

2025-10-04 14:07:08,888 [WARNING] [warnings.py:110] py.warnings: /usr/local/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 3621.
  warnings.warn(

2025-10-04 14:07:08,893 [WARNING] [warnings.py:110] py.warnings: /usr/local/lib/python3.11/site-packages/Bio/PDB/StructureBuilder.py:100: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 3942.
  warnings.warn(



+ Showing 4WZV.pdb



You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
# @title **Parameterise protein with Gasteiger charges**

# Solving the altLoc issue using split_alt_confs.py

Target_protein = '4WZV_prot' #@param {type:"string"}
output_pdb_path = os.path.join(PRT_FLD, Target_protein + "_single_conf.pdb")
cleaned_pdb_file_path = os.path.join(PRT_FLD, Accession_ID + "_prot.pdb")

!/usr/local/MGLToolsPckgs/AutoDockTools/Utilities24/prepare_pdb_split_alt_confs.py -r {cleaned_pdb_file_path} -o {output_pdb_path}

@@returning  ['A', 'C', 'B']  @@


In [ ]:
# @markdown Enter the ** ( Target Protein + _prot ) ** to be parameterised.\
# @markdown Generate a **`protein.pdbqt`** file after:
# @markdown + Removal of non-polar hydrogens, lone pair electron
# @markdown + Addition of polar hydrogens
# @markdown + Addition of Gasteiger Partial Charge

Target_protein = '4WZV_prot' #@param {type:"string"}

PROT_pdb = Target_protein + '.pdb'
PROT_pdbqt = Target_protein + '.pdbqt'
PROT_pdb_pFile = os.path.join(PRT_FLD, PROT_pdb)
PROT_pdb_dFile = os.path.join(DCK_FLD, PROT_pdb)
PROT_pdbqt_dFile = os.path.join(DCK_FLD, PROT_pdbqt)

!prepare_receptor4.py -r {PROT_pdb_pFile} -o {PROT_pdbqt_dFile} -A hydrogens -U nphs_lps -v
shutil.copy(PROT_pdb_pFile, PROT_pdb_dFile)

print(f'+ Parameterisation: {PROT_pdb} > {PROT_pdbqt}')
print(f'+ {PROT_pdbqt} > DOCKING folder')
print(f'+ {PROT_pdb} > DOCKING folder')

# Another method
# !pdb2pqr30 --ff AMBER --keep-chain --titration-state-method propka --with-ph 7.4 {PROT_pdb_pFile} {PROT_pdbqt_dFile}
# !prepare_receptor4.py -r {PROT_pdb_pFile} -o {PROT_pdbqt_dFile} -C -U nphs_lps -v

set verbose to  True
read  /content/MMP9_4WZV_1/PROTEIN/4WZV_prot.pdb
WARNING! 4WZV_prot has 225  alternate location atoms!
Use prepare_pdb_split_alt_confs.py to create pdb files containing a single conformation.

setting up RPO with mode= automatic and outputfilename=  /content/MMP9_4WZV_1/DOCKING/4WZV_prot.pdbqt
charges_to_add= gasteiger
delete_single_nonstd_residues= None
adding gasteiger charges to peptide
+ Parameterisation: 4WZV_prot.pdb > 4WZV_prot.pdbqt
+ 4WZV_prot.pdbqt > DOCKING folder
+ 4WZV_prot.pdb > DOCKING folder


In [ ]:
#@title **Generate  experimental ligand (co-crystallized ligand) PDB file**
#@markdown Enter the **< Keyword >** assigned to query for the experimental
#@markdown ligand in the **`pdb`** fle.

Keyword = "E40" #@param {type:"string"}
Elig_pdb = Keyword + ".pdb"
Elig_pdb_Efile = os.path.join(EXP_FLD, Elig_pdb)

selected = [Keyword]
with open(Elig_pdb_Efile, "w") as o, open(PDB_pdb_pFile, "r") as i:
    for line in i:
        row = line.split()
        if any(a in row for a in selected):
            o.write(line)

print(f"+ Experimental ligand extracted: {PDB_pdb} --> {Elig_pdb}")

+ Experimental ligand extracted: 4WZV.pdb --> E40.pdb


In [ ]:
#@title **Extract experimental ligands**
#@markdown This generate **clean** experimental ligands for comparison.\
#@markdown The experimental ligands will be extracted into separate files.

seperate_chain(Elig_pdb_Efile)

+ Chains detected: 2 (A, B)
+ E40_A.pdb ==> EXPERIMENTAL folder
+ E40_B.pdb ==> EXPERIMENTAL folder


In [ ]:
#@title **Display 3D ligand** {run: "auto"}
#@markdown Enter the **< Experimental Ligand >** to be viewed.

View = "E40" #@param {type:"string"}
Show_atom_labels = False #@param {type:"boolean"}
View_outline = False #@param {type:"boolean"}
eview_Efile = os.path.join(EXP_FLD, f"{View}.pdb")

VIEW_ELIG(inputE=eview_Efile,
          showAtomLabel=Show_atom_labels,
          outline=View_outline)

+ Showing E40.pdb



You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
#@title **Choose exprimental ligand**
#@markdown Enter the **< Target Experimental Ligand >** for later comparison.

Target_experimental_ligand = "E40" #@param {type:"string"}
EXP_pdb = Target_experimental_ligand + ".pdb"
EXP_pdb_Efile = os.path.join(EXP_FLD, EXP_pdb)
EXP_pdb_Dfile = os.path.join(DCK_FLD, EXP_pdb)

with Hide():
    shutil.copy(EXP_pdb_Efile, EXP_pdb_Dfile)

print(f"+ {EXP_pdb} ==> DOCKING folder")

+ E40.pdb ==> DOCKING folder


# 3. **Preparing the Ligand for Docking**
1. Enter the SMILES(2D) of Ligand using online DB (e.g., PubChem) (mol2 or sdf)
2. Addition of the Hydrogene
3. Energy Minimization (Geometrical Optimization: Force field Selection..)
  -  GAFF: ligand preparation in protein-ligand docking with compatible with AMBER
  - UFF: Metal-containing molecules, broad applicability
  - Ghemical: Educational of fast preliminary calulcations
  - MMFF94: Ligand energy minimization, structure refinement
  - MMFF94s: Precise ligand geometry optimization
4. Assign Protonation States (pH 7.4)
5. Addition of the Charge
6. Removal of the Non-Polar Hydrogenes and Ione Pairs
7. Convert 'SMILES(2D) -> MOL2 -> PDBQT(3D)

   - NOTE: The prepare_ligand4.py does not recognize the file location in for some reason. Hence, the mol2 file was generated in /content/ and then move back into LIGAND folder after parameterization.

In [ ]:
#@title **Provide chemical library**
#@markdown Upload a **< Compound Library CSV >**  onto LIGAND folder.\
#@markdown The file **should** contain the below informations:
#@markdown + **`ID`** : The identity assigned on compound, spaces are not recommended
#@markdown + **`SMILES`** : The canonical / isometric SMILES notation

Compoud_library = "ligand.csv" #@param {type:"string"}

CL_csv_Lfile = os.path.join(LIG_FLD, Compoud_library)
ligDF = pd.read_csv(CL_csv_Lfile, dtype = str)[["ID", "SMILES"]]

In [ ]:
#@title **Generate SMILES files**
#@markdown This generate **`smi`** file for each ligands in the virtual library.

lig_IDs = tuple(ligDF["ID"])
lig_SMILESs = tuple(ligDF["SMILES"])

for N,ID in enumerate(lig_IDs):
    with open(os.path.join(LIG_FLD, f"{ID}.smi"), "w") as o:
        o.write(lig_SMILESs[N])

print(f"+ {N + 1} ligand.smi ==> LIGAND folder")

+ 1 ligand.smi ==> LIGAND folder


In [ ]:
#@title **Dispay 3D ligand** {run: "auto"}
#@markdown Enter the **< Ligand >** to be viewed.

View = "Asiatic-acid" #@param {type:"string"}
Show_atom_labels = False #@param {type:"boolean"}
Show_hydrogens = False #@param {type:"boolean"}
View_outline = False #@param {type:"boolean"}
lview_Lfile = os.path.join(LIG_FLD, f"{View}.smi")

VIEW_LIG(inputL=lview_Lfile,
         showHs=Show_hydrogens,
         showAtomLabel=Show_atom_labels,
         outline=View_outline)

+ Showing Asiatic-acid.smi



You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
#@title **Parameterize ligand(s) with Gasteiger charges**
#@markdown Select **< Force Field >** for energy minimization.
#@markdown Select **< Convergence Criteria >** to set minimum iteration difference required before stopping. **[ Default is 0.00001 ]** \
#@markdown Select **< Maximum Steps >** to set maximum iterations allowed for minimisation. **[ Default is 10000 ]**
#@markdown This generate **`ligand.mol2`** and **`ligand.pdbqt`** file after:
#@markdown + Energy minization at 10,000 iterations
#@markdown + Addition of Gasteiger Partial Charge
#@markdown + Removal of non-polar hydrogens

# NOTE: The prepare_ligand4.py does not recognise the file location in for some
#       reason. Hence, the mol2 file was generated in /content/ and then move
#       back into LIGAND folder after parameterization.

Select_force_field = "MMFF94s" #@param ["GAFF", "Ghemical", "MMFF94", "MMFF94s", "UFF"]
Convergence_criteria = '0.001' # @param ['0.1', '0.01','0.001', '0.0001', '0.00001', '0.000001', '0.0000001']
Maximum_steps = 1000 # @param {type: 'slider', min:1000, max:100000, step:1000}

for N,ID in enumerate(tqdm(lig_IDs)):
  lig_Dfld = os.path.join(DCK_FLD, f"{ID}")
  os.makedirs(lig_Dfld, exist_ok=True)
  smi_file = os.path.join(LIG_FLD, f"{ID}.smi")
  mol2_file = os.path.join(DIR, f"{ID}.mol2")
  pdbqt_file = os.path.join(DCK_FLD, f"{ID}/{ID}.pdbqt")


!obabel $smi_file -O $mol2_file --gen3d --best --canonical --minimize \
--ff $Select_force_field --steps {Maximum_steps} --crit {Convergence_criteria} --sd
!prepare_ligand4.py -l $mol2_file -o $pdbqt_file -U nphs_lps -v
shutil.move(mol2_file, LIG_FLD)

print(f"+ {N+1} ligand.mol2 ==> LIGAND folder")
print(f"+ {N+1} ligand.pdbqt ==> DOCKING folder")

  0%|          | 0/1 [00:00<?, ?it/s]

Initial conformer does not pass filter!
Initial conformer count: 0
20000 attempts,  0 duplicates, 20000 failed filter.

=====> Starting conformers search with a Genetic Algorithm <=====
Perform elitist generation replacement with mutation only
Mutation probability: 0.2
Will stop after 5 generations without improvement.

The current score is not a number, will not continue.

GetConformers:
*** Open Babel Warning  in CorrectStereoAtoms
  Could not correct 2 stereocenter(s) in this molecule ()
  with Atom Ids as follows: 4 12


# **4. Setting Up Molecular Docking**
  1. Set up Grid box
    - LaBOX = Utilize the min and max XYZ coordinates of (s).LaBOX)
    - eBoxSize = Utilize the radius of gyration of oxsize)
    - eBoxSize-modifed = Utilize eBoxSize but box center is calculated with LABOGRID method
    - Autodock-Grid = 20-30 Å
    - manual = Use the slider below to define grid box

  2. Define search space on target pro box (often center of binding, active of allosteric site)
    - refer the information of web docking tool "PrankWeb"
    - refer the binding sites of Experimental ligand
  
  3. Export Coinfig File for AutoDock Vina
  



In [ ]:
#@title **Place gridbox at binding site** { run: "auto" }
#@markdown Place the **< Gridbox >** at the center of binding site. \

Focus_residues = "" #@param {type:"string"}
Method = "Autodock-Grid" #@param ["LaBOX", "eBoxSize", "eBoxSize-modified", "Autodock-Grid", "manual"]

if Method == "LaBOX":
    atomCoord = get_atom_coordinate(EXP_pdb_Dfile)
    grid = labox(atomCoord)
    Center, Size = grid[0], grid[1]

if Method == "eBoxSize":
    atomCoord = get_atom_coordinate(EXP_pdb_Dfile)
    grid = eboxsize(atomCoord)
    Center, Size = grid[0], grid[1]

if Method == "eBoxSize-modified":
    atomCoord = get_atom_coordinate(EXP_pdb_Dfile)
    grid = eboxsize_mod(atomCoord)
    Center, Size = grid[0], grid[1]

if Method == "Autodock-Grid":
    atomCoord = get_atom_coordinate(EXP_pdb_Dfile)
    grid = autodock_grid(atomCoord)
    Center, Size = grid[0], grid[1]

#@markdown **Manual Mode**
if Method == "manual":
    X = 16 # @param {"type":"slider","min":-200,"max":200,"step":1}
    Y = 44 # @param {"type":"slider","min":-200,"max":200,"step":1}
    Z = 50 # @param {"type":"slider","min":-200,"max":200,"step":1}
    Width = 22 #@param {type:"slider", min:0, max:30, step:1}
    Height = 20 #@param {type:"slider", min:0, max:30, step:1}
    Depth = 20 #@param {type:"slider", min:0, max:30, step:1}
    Scale_factor = 1.2 #@param {type:"slider", min:0.1, max:5.0, step:0.1}
    Center = [X, Y, Z]
    Size = [Width*Scale_factor, Height*Scale_factor, Depth*Scale_factor]

VIEW_GRID(inputP=PROT_pdb_dFile,
          inputE=EXP_pdb_Dfile,
          focusRes=Focus_residues,
          center=Center,
          size=Size)

+ Center: X 1.242  Y 3.453  Z 26.674
+ Size: W 22.5  H 22.5  D 22.5
+ Showing 4WZV_prot.pdb
+ Showing E40.pdb



You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
#@title **Generate docking config file**
#@markdown This generate **Config File** for AutoDock Vina.

CONF = "config_file"
CONF_Dfile = os.path.join(DCK_FLD, CONF)

with open(CONF_Dfile, "w") as o:
    o.write(f"center_x = {Center[0]} \n")
    o.write(f"center_y = {Center[1]} \n")
    o.write(f"center_z = {Center[2]} \n")
    o.write(f"\n")
    o.write(f"size_x = {Size[0]} \n")
    o.write(f"size_y = {Size[1]} \n")
    o.write(f"size_z = {Size[2]} \n")

print(f"+ {CONF} ==> DOCKING folder" )

+ config_file ==> DOCKING folder


# **5. Running AutoDock Vina**
  - Input:
    - --receptor arg: rigid part of the receptor (PDBQT)
    - --flex arg: flexible side chainds, if any (PDBQT) / refer the journals, B-factor (PDB), MD simulations..
    - --ligand arg: ligand (PDBQT)
    - --batch arg: batch ligand (PDBQT)
    - --scroing arg (=vina): scoring function (ad4, vina or vinardo)
  - Search space (required):
    - --maps arg: affinity maps for the autodock4.2 (ad4) or vina
    - --center_x, _y, _z: terms coodinate of the center (Angstrom)
    - --size_x, _y, _z: size in the dimension (Angstrom)
    - --autobox: set maps dimensions base on input ligand(s) for --score_only and --local_only
  - Output:
    - --out arg: outptu models (PDBQT) the defult is chosen based on the ligand file name
    - --dir arg: output directory for batch mode
    - --write_maps arg: output filename (directory + prefix name) for maps. Option --force_even_voxels may be need to comply with .map foramat
  - Misc:
    - --cpu arg (=0): the number of CPUs to use (the default is to try to detect the number of CPUs or, falinng, use 1)
    - --seed arg (=0) explicit random seed
    - --exhaustiveness arg (=8) exhaustiveness of the global search (roughly proportional to time): 1+
    - --max_evals arg (=0) number of evaluations in each MC run (if zero, which is the default, the number of MC steps is based on heuristics) / MC = Monte Carlo method
    - --num_modes arg (=9) maximum number of binding modes to generate
    - --min_rmsd arg (=1) minimum RMSD between output poses
    - --energy_range arg (=3) maximum energy difference between the best binding mode and the worst one displayed (kcal/mol)
    - --spacing arg (=0.375) grid spacing (Angstrom)
    - --verbosity arg (=1) verbosity (0=no output, 1=normal, 2=verbose)

  - Configuration file (optional):
    - --config arg the above options can be put here

In [ ]:
#@title **Run AutoDock Vina**
#@markdown Select **< Exhaustiveness >** for molecular docking. (Default: 8)\
#@markdown Autodock Vina will performs molecular docking on targeted protein.\
#@markdown This generate **`ligand_output.pdbqt`**.

#@markdown **NOTE:** Compound with many rotatable bonds may required higher
#@markdown level of exhaustiveness to ensure accurate and consistent pose
#@markdown prediction.
import timeit
import contextlib
Exhaustiveness = 8 #@param {type:"slider", min:1, max:128, step:1}
ex = Exhaustiveness
cpu_num = os.cpu_count()
time_elp = N = 0

for N,ID in enumerate(tqdm(lig_IDs)):
    pdbqt_Dfile = os.path.join(DCK_FLD, f"{ID}/{ID}.pdbqt")
    log_txt_Dfile = os.path.join(DCK_FLD, f"{ID}/{ID}_log.txt")
    out_pdbqt_Dfile = os.path.join(DCK_FLD, f"{ID}/{ID}_output.pdbqt")

    # -- Start docking --
    start = timeit.default_timer()
    with Hide():
        with open(log_txt_Dfile, "w") as i:
            with contextlib.redirect_stdout(i):
                %vina --receptor $PROT_pdbqt_dFile --ligand $pdbqt_Dfile \
                --out $out_pdbqt_Dfile --config $CONF_Dfile \
                --exhaustiveness $ex --cpu $cpu_num --verbosity 2
        with open(log_txt_Dfile, "r") as r:
            data = r.read().splitlines(True)
        with open(log_txt_Dfile, "w") as o:
            o.writelines(data[1:])
    stop = timeit.default_timer()
    # -- End docking --

    runtime = stop - start
    time_elp += runtime
    avg = int(time_elp/(N+1))
    remain = int(avg*(len(lig_IDs)-N)-avg)

    RTO = f"{np.round(runtime, 2)}s"
    RO = f"{remain//3600:02}:{remain%3600//60:02}:{remain%60:02}"
    print(f"{ID:^10} {f'{RTO} taken':^20}  {f'{RO} remaining':^24}")

print(f"")
print(f"+ Operation completed")
print(f"+ Time elapsed: {int(time_elp//60):02}m {int(time_elp%60):02}s")


  0%|          | 0/1 [00:00<?, ?it/s]

Madecassic-acid     0.11s taken          00:00:00 remaining   

+ Operation completed
+ Time elapsed: 00m 00s


In [ ]:
#@title **Process output file**
#@markdown This generate **`ligand_output.sdf`** for comparison with experimental ligand.\
#@markdown This also generate **`ligand.pdb`** for each binding modes.

pose_count = 0
for N,ID in enumerate(tqdm(lig_IDs)):
    out_pdbqt_Dfile = os.path.join(DCK_FLD, f"{ID}/{ID}_output.pdbqt")
    out_sdf_Dfile = os.path.join(DCK_FLD, f"{ID}/{ID}_output.sdf")
    out_d_pdb_Dfile = os.path.join(DCK_FLD, f"{ID}/{ID}_.pdb")
    input = pybel.readfile("pdbqt", out_pdbqt_Dfile)
    output = pybel.Outputfile("sdf", out_sdf_Dfile, True)

    mols = [m for m in input]
    for mol in mols:
        mol.data.update({"POSE": mol.data["MODEL"]})
        mol.data.update({"SCORE": mol.data["REMARK"].split()[2]})
        mol.data.update({"RMSD_LB_BP": mol.data["REMARK"].split()[3]})
        mol.data.update({"RMSD_UB_BP": mol.data["REMARK"].split()[4]})
        del mol.data["MODEL"], mol.data["REMARK"], mol.data["TORSDO"]
        output.write(mol)
    output.close()

    with Hide():
        !obabel $out_pdbqt_Dfile -O $out_d_pdb_Dfile -m

    lig_files = os.listdir(os.path.join(DCK_FLD, ID))
    pose_count += len(lig_files)-4

print(f"+ {N+1} ligand_output.sdf ==> DOCKING folder")
print(f"+ {pose_count} ligand_n.pdb ==> DOCKING folder")

  0%|          | 0/1 [00:00<?, ?it/s]

OSError: No such file: '/content/MMP9_4WZV_1/DOCKING/Madecassic-acid/Madecassic-acid_output.pdbqt'

In [ ]:
#@title **Generate protein-ligand complex file**
#@markdown This merge ligand pose and protein into a **`PL.pdb`** file. \
#@markdown Required by **PLIP** to profile binding interactions.

N = 0
for ID in tqdm(lig_IDs):
    lig_files = os.listdir(os.path.join(DCK_FLD, ID))
    lig_Ifld = os.path.join(INT_FLD, f"{ID}")
    os.makedirs(lig_Ifld, exist_ok=True)

    for n in range(len(lig_files)-4):
        pose_pdb_Dfile = os.path.join(DCK_FLD, f"{ID}/{ID}_{n+1}.pdb")
        complex_pdb_Ifile = os.path.join(INT_FLD, f"{ID}/{ID}_{n+1}_PL.pdb")
        prot_data = open(PROT_pdb_dFile, "r").readlines()
        pose_data = open(pose_pdb_Dfile, "r").readlines()

        with open(complex_pdb_Ifile, "w") as o:
            for line in prot_data[:-1]:
                o.write(line)
            for line in pose_data:
                word = line.split()
                if word[0] == "ATOM" or word[0] == "CONECT":
                    o.write(line)
            o.write("END")
        N += 1

print(f"+ {N} PL.pdb ==> INTERACTION folder")

  0%|          | 0/1 [00:00<?, ?it/s]

+ 9 PL.pdb ==> INTERACTION folder


In [ ]:
#@title **Profile protein-ligand interactions**
#@markdown This identify **binding interactions** between docked ligand and target protein. \
#@markdown All interactions data is exported as **`inter.csv`** file.

N = 0
for ID in tqdm(lig_IDs):
    lig_files = os.listdir(os.path.join(DCK_FLD, ID))
    print(lig_files)
    for n in range(len(lig_files)-4):
        complex_pdb_Ifile = os.path.join(INT_FLD, f"{ID}/{ID}_{n+1}_PL.pdb")
        interaction_profiler(complex_pdb_Ifile)
        N += 1

print(f"+ {N} inter.CSV ==> INTERACTION folder")

  0%|          | 0/1 [00:00<?, ?it/s]

['Ethacrynic-Acid_2.pdb', 'Ethacrynic-Acid_5.pdb', 'Ethacrynic-Acid_8.pdb', 'Ethacrynic-Acid_output.sdf', 'Ethacrynic-Acid_1.pdb', 'Ethacrynic-Acid_7.pdb', 'Ethacrynic-Acid_4.pdb', 'Ethacrynic-Acid_9.pdb', 'Ethacrynic-Acid_3.pdb', 'Ethacrynic-Acid.pdbqt', 'Ethacrynic-Acid_output.pdbqt', 'Ethacrynic-Acid_log.txt', 'Ethacrynic-Acid_6.pdb']
+ 9 inter.CSV ==> INTERACTION folder


In [ ]:
pose_count = 0
for ID in tqdm(lig_IDs):
    lig_files = os.listdir(os.path.join(DCK_FLD, ID))
    pdb_files = sorted([f for f in lig_files if f.endswith(".pdb") and f"{ID}_" in f])

    for n, pdb_file in enumerate(pdb_files):
        complex_pdb_Ifile = os.path.join(INT_FLD, f"{ID}/{ID}_{n+1}_PL.pdb")
        interaction_profiler(complex_pdb_Ifile)
        pose_count += 1

print(f"+ {pose_count} inter.CSV ==> INTERACTION folder")


  0%|          | 0/1 [00:00<?, ?it/s]

+ 9 inter.CSV ==> INTERACTION folder


# 7. **Results of Docking**
  - Display 3D binding pose
  - Generate the Scroing Results
  - Rank docking scores
    - RMSD_EL = RMSD vs Exp. Lig.
    - RMSD_LB_BP = RMSD Lower Bound vs Best Pose
    - RMSD_LB_UP = RMSD Upper Bound vs Best Pose

In [ ]:
#@title **Generate docking results**
#@markdown This generate **scoring result** of docked ligands in **`csv`** format.

EXP_pose = Chem.MolFromPDBFile(EXP_pdb_Dfile)
EXP_pose.SetProp("_Name", os.path.basename(EXP_pdb_Dfile)[:-4])

for N,ID in enumerate(tqdm(lig_IDs)):
    out_sdf_Dfile = os.path.join(DCK_FLD, f"{ID}/{ID}_output.sdf")
    rank_csv_Dfile = os.path.join(DCK_FLD, f"{ID}/{ID}_result.csv")
    data = pd.DataFrame(get_score(out_sdf_Dfile, EXP_pose, result="full"))
    data.to_csv(rank_csv_Dfile, index=False)

print(f"+ {N+1} result.csv ==> DOCKING folder")

  0%|          | 0/1 [00:00<?, ?it/s]

+ 1 result.csv ==> DOCKING folder


In [ ]:
#@title **Rank docking scores**
#@markdown This cluster and rank **scoring results** in ascending order. \
#@markdown The overall ranking is exported as **`ranked_result.csv`**. \
#@markdown **Top 10** results are shown below. \

#@markdown Noted that:
#@markdown + **`RMSD_EL`** = RMSD vs Exp. Lig.
#@markdown + **`RMSD_LB_BP`** = RMSD Lower Bound vs Best Pose
#@markdown + **`RMSD_LB_UP`** = RMSD Upper Bound vs Best Pose

vina_rank = pd.DataFrame()
for N,ID in enumerate(lig_IDs):
    out_sdf_Dfile = os.path.join(DCK_FLD, f"{ID}/{ID}_output.sdf")
    vina_rank = pd.concat([vina_rank, get_score(out_sdf_Dfile, EXP_pose, result="first")])

vina_rank = vina_rank.sort_values("SCORE", ascending=True, ignore_index=True)
rank_csv_Dfile = os.path.join(DCK_FLD, "ranked_result.csv")
vina_rank.to_csv(rank_csv_Dfile)

print(f"+ 1 ranked_result.xlsx ==> DOCKING folder")
vina_rank

+ 1 ranked_result.xlsx ==> DOCKING folder


,ID,SCORE,RMSD_EL
0,Ethacrynic-Acid_1,-6.227,11.114


In [ ]:
#@title **Show docking result of ligand** {run : "auto"}

Select_ligand = "Ethacrynic-Acid" #@param {type:"string"}
LIG_out_csv_Dfile = os.path.join(DCK_FLD, f"{Select_ligand}/{Select_ligand}_result.csv")
ligResultDF = pd.read_csv(LIG_out_csv_Dfile)
ligResultDF

,ID,POSE,SCORE,RMSD_EL,RMSD_LB_BP,RMSD_UB_BP
0,Ethacrynic-Acid_1,1,-6.227,11.114,0.000,0.000
1,Ethacrynic-Acid_2,2,-6.031,4.758,4.151,5.185
2,Ethacrynic-Acid_3,3,-5.992,5.597,8.177,10.494
3,Ethacrynic-Acid_4,4,-5.855,4.688,8.631,10.904
4,Ethacrynic-Acid_5,5,-5.833,4.131,8.278,9.736
5,Ethacrynic-Acid_6,6,-5.729,4.179,4.486,5.606
6,Ethacrynic-Acid_7,7,-5.654,4.857,9.012,11.525
7,Ethacrynic-Acid_8,8,-5.605,4.960,8.818,11.297
8,Ethacrynic-Acid_9,9,-5.522,5.538,4.635,5.362


In [ ]:
#@title **Show interaction profile of ligand** {run: "auto"}

Select_ligand = "Ethacrynic-Acid_2" #@param {type : "string"}
LIG_inter_CSV_Ifile = os.path.join(INT_FLD, f"{Select_ligand[:-2]}/{Select_ligand}_inter.csv")
view_interaction(LIG_inter_CSV_Ifile, result="summary")

,RESNR,RESTYPE,DIST_CALC,BOND,COLOR
0,1007,PHE,3.73,HYDROPHOBIC,GREEN
1,1007,PHE,3.49,HYDROPHOBIC,GREEN
2,1010,VAL,3.94,HYDROPHOBIC,GREEN
3,1032,VAL,3.98,HYDROPHOBIC,GREEN
4,1044,PHE,3.67,HYDROPHOBIC,GREEN
5,1007,PHE,3.90,HBOND,LIGHT BLUE
6,1136,ARG,2.84,HBOND,LIGHT BLUE


In [ ]:
#@title **Display 3D binding mode** { run: "auto" }

#@markdown **PROTEIN MODEL**
View_protein = "4XLV_prot" #@param {type:"string"}
Model = "cartoon" #@param ["none", "line", "stick",  "cartoon"]
Model_colour = "white" #@param ["red", "orange", "yellow", "green", "blue", "violet", "purple", "white"]
Focus_residues = "" #@param {type:"string"}
Residues_colour = "white" #@param ["red", "orange", "yellow", "green", "blue", "violet", "purple", "white"]
Show_all_subunits = False #@param {type:"boolean"}
Show_residues_label = True #@param {type:"boolean"}
Show_interaction = True #@param {type:"boolean"}
Show_VDW_surface = True #@param {type:"boolean"}
View_outline = False #@param {type:"boolean"}

#@markdown ---

#@markdown **LIGAND MODEL** \
#@markdown Enter the **< Ligand >** to be viewed.
View_ligand = "Ethacrynic-Acid_2" #@param {type:"string"}
View_experimental_ligand = "ACP" #@param {type:"string"}
Show_ligand = True #@param {type:"boolean"}
Show_experimental_ligand = True #@param {type:"boolean"}
Show_all_pose = False #@param {type:"boolean"}
Show_best_pose = False #@param {type:"boolean"}

#@markdown ---

#@markdown **PARTNER PROTEIN MODEL** \
#@markdown (OPTIONAL, upload to DOCKING FOLDER) \
#@markdown Enter the **< Partner Protein>** to be view.

View_partner_protein = "" #@param {type:"string"}
Show_partner_protein = False #@param {type:"boolean"}

pview_Dfile = os.path.join(DCK_FLD, f"{View_protein}.pdb")
ppview_Dfile = os.path.join(DCK_FLD, f"{View_partner_protein}.pdb")
lview_Dfile = os.path.join(DCK_FLD, f"{View_ligand[:-2]}/{View_ligand}.pdb")
eview_Dfile = os.path.join(DCK_FLD, f"{View_experimental_ligand}.pdb")
iview_Ifile = os.path.join(INT_FLD, f"{View_ligand[:-2]}/{View_ligand}_inter.csv")

VIEW_PILE(inputP=pview_Dfile,
          inputPP=ppview_Dfile,
          inputL=lview_Dfile,
          inputE=eview_Dfile,
          inputCSV=iview_Ifile,
          model=Model,
          color=Model_colour,
          focusRes=Focus_residues,
          focusResColor=Residues_colour,
          showInter=Show_interaction,
          showSubunit=Show_all_subunits,
          showResLabel=Show_residues_label,
          showVDW=Show_VDW_surface,
          showPartProt=Show_partner_protein,
          showExpLig=Show_experimental_ligand,
          showLig=Show_ligand,
          showAllPose=Show_all_pose,
          showBestPose=Show_best_pose,
          outline=View_outline)

2025-10-03 08:52:42,319 [WARNING] [warnings.py:110] py.warnings: /usr/local/lib/python3.11/site-packages/Bio/PDB/PDBParser.py:384: PDBConstructionWarning: Ignoring unrecognized record 'TER' at line 2398
  warnings.warn(



+ Showing 4XLV_prot.pdb
+ Showing ACP.pdb (gray)
+ Showing Ethacrynic-Acid_2.pdb (orange)



You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [ ]:
#@title Downloading Files to Local Directory

import shutil
import os
from google.colab import files

# Specify the directory path for downloading.
folder_to_download = Job_name

# Specify the name of the compressed file
archive_name = Job_name
output_filename = shutil.make_archive(archive_name, 'zip', folder_to_download)

# Download the compressed file
files.download(output_filename)

print(f"'{folder_to_download}' folder to '{output_filename}'.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'IGF1_4XLV_1' folder to '/content/IGF1_4XLV_1.zip'.


In [ ]:
#@title Downloading Files to Googld Drive

import shutil
from google.colab import files
import os

# Google Drive mount
drive.mount('/content/drive')

# Use the Job_name variable for the folder path
folder_path = os.path.join('/content', Job_name)

# Use the Job_name variable for the zip file name
zip_dest_path = os.path.join('/content/drive/MyDrive', f"{Job_name}.zip")

# Use the Job_name variable for the temporary zip file name
zip_temp_path = os.path.join('/content', f"{Job_name}.zip")

if os.path.exists(folder_path):

    shutil.make_archive(zip_temp_path.replace('.zip', ''), 'zip', folder_path)
    shutil.move(zip_temp_path, zip_dest_path)

    print(f"'{folder_path}' -> '{zip_dest_path}'.")

else:
    print(f"Not finding : {folder_path}")

MessageError: Error: credential propagation was unsuccessful